In [ ]:
import datacube
import xarray as xr
import rioxarray
import dask
from dask.distributed import performance_report
from dask.distributed import Client

#%matplotlib inline
#import hvplot.xarray

In [ ]:
## Connect to ODC
dc = datacube.Datacube()

## Start local Dask cluster
## https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster
n_workers = 1
n_threads = 24
client = Client(processes=False, 
                n_workers=n_workers, 
                threads_per_worker=n_threads)

## Define some other variables 
## (Only done here to use them in the output filenames)
orbit = "desc"
chunk_x = 1500
chunk_y = 1500

In [ ]:
"""
Subset for testing: 
x=(11.5, 11.8)
y=(50.6, 51)
"""

ds = dc.load(product=f"s1_ARD_{orbit}",
             dask_chunks={'time':-1, 'y':chunk_y, 'x':chunk_x})

vh = ds.VH
vh

In [ ]:
vh_count = xr.where(vh > -99, True, False).sum(dim="time")
vh_count

In [ ]:
%%time
with performance_report(filename=f"./reports/obs_perpixel_SAR_{orbit}__{n_workers}-{n_threads}_{chunk_x}-{chunk_y}.html"):
    vh_count = vh_count.compute()

In [ ]:
vh_count

In [ ]:
vh_count.rio.to_raster(f"./output/obs_perpixel_SAR_{orbit}__{n_workers}-{n_threads}_{chunk_x}-{chunk_y}.tif", dtype="float32")